In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
# Load a pre-trained MobileNetV2 SSD model from TensorFlow Hub
import tensorflow_hub as hub

In [ ]:
# Load the model
model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"
detector = hub.load(model_url)


In [ ]:
# Load labels for COCO dataset
labels_path = tf.keras.utils.get_file('coco_labels.txt',
    'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt')
with open(labels_path, 'r') as f:
    labels = f.read().splitlines()

702/702 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step


In [ ]:
# Load and preprocess the image
def load_image(image_path):
    # Check if the image file exists
    if not cv2.haveImageReader(image_path):
        raise FileNotFoundError(f"Image file not found or unsupported format: {image_path}")

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(image_rgb, (320, 320))
    resized = tf.expand_dims(resized, 0)
    return image, resized

In [ ]:
# Perform detection
def detect_objects(image_path):
    image, input_tensor = load_image(image_path)
    results = detector(input_tensor)
    result = {key: value.numpy() for key, value in results.items()}

    # Draw boxes
    for i in range(len(result['detection_scores'])):
        score = result['detection_scores'][i]
        if score < 0.5:
            continue
        box = result['detection_boxes'][i]
        class_id = int(result['detection_classes'][i])
        label = labels[class_id]

        h, w, _ = image.shape
        y_min, x_min, y_max, x_max = box
        (left, right, top, bottom) = (int(x_min * w), int(x_max * w),
                                      int(y_min * h), int(y_max * h))

        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(image, f'{label}: {score:.2f}', (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# Example usage
detect_objects("sample.jpg")

FileNotFoundError: Image file not found or unsupported format: sample.jpg